In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.nn import ChebConv
from torch_geometric.data import Data, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load your time series sensor data into a pandas DataFrame
df = pd.read_csv(r'C:\Users\SUJAN\preprocessed_data.csv')

# Exclude the 'date' and 'machine_status' columns from the DataFrame
df = df.iloc[:, :-2]

# Split the data into train and test sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Convert the train and test data to PyTorch tensors
train_data = torch.tensor(train_data.values, dtype=torch.float32)
test_data = torch.tensor(test_data.values, dtype=torch.float32)

# Define the GNN model
class GNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(GNN, self).__init__()
        self.conv1 = ChebConv(input_dim, hidden_dim, K=2)
        self.conv2 = ChebConv(hidden_dim, input_dim, K=2)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Create a PyTorch geometric Data object for the train data
train_edges = torch.tensor([[i, i + 1] for i in range(0, len(train_data) - 1)], dtype=torch.long).t().contiguous()
train_graph = Data(x=train_data, edge_index=train_edges)

# Create a PyTorch geometric Data object for the test data
test_edges = torch.tensor([[i, i + 1] for i in range(0, len(test_data) - 1)], dtype=torch.long).t().contiguous()
test_graph = Data(x=test_data, edge_index=test_edges)

# Create a PyTorch geometric DataLoader for batch training
train_loader = DataLoader([train_graph], batch_size=1, shuffle=True)

# Initialize the GNN model and define the optimizer
model = GNN(input_dim=len(df.columns), hidden_dim=64)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the GNN model
model.train()
for epoch in range(100):
    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = F.mse_loss(out[:-1], batch.x[1:])  # Mean Squared Error loss
        loss.backward()
        optimizer.step()

# Test the GNN model on the test data
model.eval()
with torch.no_grad():
    test_output = model(test_data, test_graph.edge_index)

# Compute the anomaly scores for each variable
anomaly_threshold = 0.1  # Adjust this threshold as needed
anomaly_mask = torch.abs(test_data - test_output) > anomaly_threshold
anomaly_scores = torch.mean(torch.abs(test_data - test_output), dim=1)

# Compute precision, recall, confusion matrix, and F1 score
y_pred = torch.argmax(test_output, dim=1)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Define the column names
column_names = ['sensor_11']

# Plot the results
plt.figure(figsize=(12, 8))
plt.plot(test_data[:, 0], label='Original Data', color='blue')
plt.plot(test_output[:, 0], label='Predicted Data', color='green')
plt.scatter(torch.nonzero(anomaly_mask[:, 0]).squeeze().tolist(), test_output[:, 0][anomaly_mask[:, 0]].squeeze().tolist(),
            color='red', marker='X', label='Anomalies')
plt.xlabel('Time')
plt.ylabel('Value')
plt.title(f'{column_names[0]}')
plt.legend()
plt.show()

# Print precision, recall, confusion matrix, and F1 score
print('Precision:', precision)
print('Recall:', recall)
print('Confusion Matrix:')
print(confusion_mat)
print('F1 Score:', f1)

NameError: name 'precision_score' is not defined